In [2]:
%reload_ext autoreload
%autoreload 2

#load libraries
import glob
import os
import pandas as pd
from datetime import datetime
import numpy as np
import main_funcs as mfun

import plot_funcs as pfun
import utils_funcs as utils # utils is from Vape - catcher file: 
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import zscore
import pickle


# Get the data info
pfun.set_figure()

# Get the data info
infoPath = 'C:/Users/Lak Lab/Documents/Github/sideBiasLateralisation/analysis/infoForAnalysis.pkl'
info = pd.read_pickle(infoPath)

## Parameters
fRate = 1000/30
responsiveness_test_duration = 1000.0 #in ms 
pre_frames    = 2000.0# in ms
pre_frames    = int(np.ceil(pre_frames/fRate))
post_frames   = 6000.0 # in ms
post_frames   = int(np.ceil(post_frames/fRate))
analysisWindowDur = 750 # in ms
analysisWindowDur = int(np.ceil(analysisWindowDur/fRate))

info.recordingList

Env: sideBias


,animalID,recordingDate,recordingID,sessionName,learningData,twoP,path,sessionNameWithPath,analysispathname,filepathname,eventTimesExtracted,eventTimesPath,variance,imagingDataExtracted,imagingDataPath
0,MBL015,2025-05-20,1,2025-05-20_1_MBL015,False,True,Z:/MBL015\2025-05-20,Z:/MBL015\2025-05-20\1\2025-05-20_1_MBL015_Blo...,C:/Users/Lak Lab/Documents/Github/sideBiasLate...,Z:/MBL015\2025-05-20\1,1.0,C:/Users/Lak Lab/Documents/Github/sideBiasLate...,NaN,1.0,C:/Users/Lak Lab/Documents/Github/sideBiasLate...
1,MBL015,2025-05-22,1,2025-05-22_1_MBL015,False,True,Z:/MBL015\2025-05-22,Z:/MBL015\2025-05-22\1\2025-05-22_1_MBL015_Blo...,C:/Users/Lak Lab/Documents/Github/sideBiasLate...,Z:/MBL015\2025-05-22\1,1.0,NaN,11.156564,1.0,C:/Users/Lak Lab/Documents/Github/sideBiasLate...


In [3]:
#Explore dynamics in each sessions
# For each recording one file saved, each index is a cell 

pd.set_option('mode.chained_assignment', None)

for ind, recordingDate in enumerate(info.recordingList.recordingDate):

   if (info.recordingList.imagingDataExtracted[ind]==1):
      try:
         print(str(ind) + ': Creating: ' + info.recordingList.analysispathname[ind])
         #Create a huge dictionary with all cells and parameters for each cell
         pathname = info.recordingList.analysispathname[ind]

         ########## Organise stimuli times 
         filenameCSV = info.recordingList.analysispathname[ind] + info.recordingList.sessionName[ind] + '_CorrectedeventTimes.csv'
         filenameCSV = [f for f in glob.glob(filenameCSV)]    
         behData     = pd.read_csv(filenameCSV[0], header=0)
         visTimes    = behData['stimulusOnsetTime']
         rewardTimes = behData['rewardTime']
         choice      = behData['choice']
         choiceTimes = behData['choiceStartTime']
         # Make a variable for rewarded trials
         rewarded    =  behData['rewardTime'].notna() # True for rewarded which also means correct response
         # Calculate reward Time for non-rewarded Trials
         choiceCompleteTimes = behData['choiceCompleteTime']
         diff_time = np.nanmean(rewardTimes -choiceTimes)
         nan_indices = np.isnan(rewardTimes) # unrewarded trials
         rewardTimes[nan_indices] = choiceCompleteTimes[nan_indices] + diff_time
        
         # Create variable stimulus that takes the non-zero value of contrastLeft or contrastRight
         stimulus = np.where(behData['contrastLeft'] != 0, behData['contrastLeft'], behData['contrastRight'])

         # Make a variable for stimType
         stimType = pd.Series([''] * len(behData['choice']))
         stimType [:] = 'Left'

         # Stimulus type Right
         idx_right = behData['correctResponse'] == 'Right'
         stimType[idx_right] = 'Right'
         
         # Stimulus type Zero
         idx_zero = behData['contrastRight'] - behData['contrastLeft'] == 0
         stimType[idx_zero] = 'Zero'

         # Get the stim start times 
         filenameTXT = os.path.join(info.recordingList.path[ind],'twoP') +'\*_imaging_frames.txt'
         filenameTXT= [f for f in glob.glob(filenameTXT)]    
         frame_clock = pd.read_csv(filenameTXT[0],  header= None)
            
         stimFrameTimes    = utils.stim_start_frame_Dual2Psetup (frame_clock, visTimes)
         rewardFrameTimes  = utils.stim_start_frame_Dual2Psetup (frame_clock, rewardTimes)
         choiceFrameTimes  = utils.stim_start_frame_Dual2Psetup (frame_clock, choiceTimes)

         ########## Organise calcium imaging traces 
         imData = pd.read_pickle (pathname +'imaging-data.pkl')
         fluR      = imData['flu']
         stat      = imData['stat']

         # clean detrended traces
         flu = utils.clean_traces(fluR)
         #flu_normalised = mfun.norm_to_zero_one (flu)
         flu_normalised = zscore (fluR)
         dffTrace_reward ={} 
         dffTrace_mean_reward ={}
         dffTrace_stimuli ={} 
         dffTrace_mean_stimuli ={}
         dffTrace_choice ={}
         dffTrace_mean_choice ={}
         dff_mean_stimuli ={}
         dff_mean_reward ={}
         dff_mean_choice ={}

         ### Get dff values for 2 time windows
         tTypesName = ['Rewarded', 'Unrewarded',              # reward type
                     'Left choices', 'Right choices',       # choice type
                     'Rewarded Left', 'Rewarded Right',     # new combined types
                     '0.0625','0.125', '0.25', '0.5'        # stimuli type
                     ]
         tTypes = [(rewarded==True),(rewarded!=True),          # reward type
                  (choice == 'Left'),(choice == 'Right'),      # choice type
                  (rewarded==True) & (choice == 'Left'),       # new combined type
                  (rewarded==True) & (choice == 'Right'),      # new combined type
                  (stimulus == 0.0625),(stimulus == 0.125), 
                  (stimulus == 0.25),(stimulus == 0.5),        # stimuli type
                  ]


         for indx, t in enumerate(tTypesName):
            # For reward
            selected_indices = [rewardFrameTimes[i] for i in np.where((tTypes[indx]==True)& (~np.isnan(rewardFrameTimes)))[0]]  
            selected_indices = [value for value in selected_indices if value == value]
            dffTrace_reward[t] = utils.flu_splitter(flu,selected_indices, pre_frames, post_frames)  # Cell x time x trial
            dffTrace_mean_reward[t] = np.mean(dffTrace_reward[t],2) if len(selected_indices)>2 else None # Cell x time
            dff_mean_reward[t] = np.nanmean(dffTrace_mean_reward[t][:, (pre_frames): (pre_frames + analysisWindowDur)],1) if len(selected_indices)>2 else None
            
            # For stimuli
            selected_indices = [stimFrameTimes[i] for i in np.where(tTypes[indx]==True)[0]]
            selected_indices = [value for value in selected_indices if value == value]
            dffTrace_stimuli[t] = utils.flu_splitter(flu, selected_indices, pre_frames, post_frames) # Cell x time x trial
            dffTrace_mean_stimuli[t] = np.mean(dffTrace_stimuli[t],2) if len(selected_indices)>2 else None # Cell x time
            dff_mean_stimuli[t] = np.nanmean(dffTrace_mean_stimuli[t][:, (pre_frames): (pre_frames + analysisWindowDur)],1) if len(selected_indices)>2 else None

            # For choice
            selected_indices = [choiceFrameTimes[i] for i in np.where((tTypes[indx]==True)& (~np.isnan(choiceFrameTimes)))[0]] 
            selected_indices = [value for value in selected_indices if value == value]
            dffTrace_choice[t] = utils.flu_splitter(flu, selected_indices, pre_frames, post_frames) # Cell x time x trial
            dffTrace_mean_choice[t] = np.mean(dffTrace_choice[t],2) if len(selected_indices)>2 else None # Cell x time
            dff_mean_choice[t] = np.nanmean(dffTrace_mean_choice[t][:, (pre_frames): (pre_frames + analysisWindowDur)],1) if len(selected_indices)>2 else None

         #### Save params
         filenameINFO = pathname + 'imaging-dffTrace.pkl'
         print('Saving: '+ filenameINFO)
         with open(filenameINFO, 'wb') as f:
            pickle.dump([dffTrace_reward,dffTrace_stimuli, dffTrace_choice] , f)
         filenameINFO = pathname + 'imaging-dffTrace_mean.pkl'
         print('Saving: '+ filenameINFO)
         with open(filenameINFO, 'wb') as f:
            pickle.dump([dffTrace_mean_reward,
                         dffTrace_mean_stimuli,
                         dffTrace_mean_choice ] , f)
         print('Saved successfully: '+ filenameINFO)
         filenameINFO = pathname + 'imaging-dff_mean.pkl'
         print('Saving: '+ filenameINFO)
         with open(filenameINFO, 'wb') as f:
            pickle.dump([dff_mean_reward,
                         dff_mean_stimuli,
                         dff_mean_choice ] , f)
         print('Saved successfully: '+ filenameINFO)


         ### Plot dynamics:# Explore reward, stimuli, choice representation
      
         pfun.set_figure()
         figsPath = info.figsPath
         selectedSession = 0
         duration = [3]
         colormap = 'viridis'
         zscoreRun = False 


         # Heat Plots
         savefigname = 'SessionComparison-heatmap-reward_' + str(duration[0]) + 'sec'
         analysis_params = ['Rewarded', 'Unrewarded']
         pfun.heatmap_sessions(dffTrace_mean_reward,analysis_params, colormap,
                              selectedSession, duration, savefigname, pathname) 
         plt.close()
                        
         savefigname = 'SessionComparison-heatmap-choice_' + str(duration[0]) + 'sec'
         analysis_params = ['Left choices','Right choices']
         pfun.heatmap_sessions(dffTrace_mean_choice,analysis_params, colormap,
                              selectedSession, duration, savefigname, pathname) 
         plt.close()

         savefigname = 'SessionComparison-heatmap-stimuli_' + str(duration[0]) + 'sec'
         analysis_params = ['0.0625','0.125', '0.25', '0.5']
         pfun.heatmap_sessions(dffTrace_mean_stimuli,analysis_params, colormap,
                              selectedSession, duration, savefigname, pathname) 
         plt.close()

         # Mean Plots 
         colormap = 'Set2'
         zscoreRun = False 

         savefigname = 'SessionComparison-mean-reward_' + str(duration[0]) + 'sec'
         analysis_params = ['Rewarded', 'Unrewarded']
         pfun.lineplot_sessions(dffTrace_mean_reward, analysis_params, colormap,
                           duration, zscoreRun, savefigname, pathname) 
         plt.close()


         savefigname = 'SessionComparison-mean-choice_' + str(duration[0]) + 'sec'
         analysis_params = ['Left choices','Right choices']
         pfun.lineplot_sessions(dffTrace_mean_choice, analysis_params, colormap,
                           duration, zscoreRun, savefigname, pathname) 
         plt.close()
         print('Completed')
         info.recordingList.loc[ind,'analysisVariableExtracted'] = 0 
      except:
         info.recordingList.loc[ind,'analysisVariableExtracted'] = 1
         print('Problem in plotting')


0: Creating: C:/Users/Lak Lab/Documents/Github/sideBiasLateralisation\analysis\2025-05-20_MBL015_1\
Saving: C:/Users/Lak Lab/Documents/Github/sideBiasLateralisation\analysis\2025-05-20_MBL015_1\imaging-dffTrace.pkl
Saving: C:/Users/Lak Lab/Documents/Github/sideBiasLateralisation\analysis\2025-05-20_MBL015_1\imaging-dffTrace_mean.pkl
Saved successfully: C:/Users/Lak Lab/Documents/Github/sideBiasLateralisation\analysis\2025-05-20_MBL015_1\imaging-dffTrace_mean.pkl
Saving: C:/Users/Lak Lab/Documents/Github/sideBiasLateralisation\analysis\2025-05-20_MBL015_1\imaging-dff_mean.pkl
Saved successfully: C:/Users/Lak Lab/Documents/Github/sideBiasLateralisation\analysis\2025-05-20_MBL015_1\imaging-dff_mean.pkl
Completed
1: Creating: C:/Users/Lak Lab/Documents/Github/sideBiasLateralisation\analysis\2025-05-22_MBL015_1\
Saving: C:/Users/Lak Lab/Documents/Github/sideBiasLateralisation\analysis\2025-05-22_MBL015_1\imaging-dffTrace.pkl
Saving: C:/Users/Lak Lab/Documents/Github/sideBiasLateralisation\a